# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770877278562                   -0.52    9.0
  2   -2.772145981830       -2.90       -1.32    1.0
  3   -2.772170409929       -4.61       -2.49    1.0
  4   -2.772170661944       -6.60       -3.26    1.0
  5   -2.772170721530       -7.22       -3.90    2.0
  6   -2.772170722983       -8.84       -5.10    1.0
  7   -2.772170723013      -10.52       -5.28    2.0
  8   -2.772170723015      -11.74       -5.69    1.0
  9   -2.772170723015      -12.44       -6.42    1.0
 10   -2.772170723015      -13.44       -7.46    2.0
 11   -2.772170723015   +  -14.57       -8.11    1.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770866849239                   -0.52    9.0
  2   -2.772063331475       -2.92       -1.32    1.0
  3   -2.772083132691       -4.70       -2.48    1.0
  4   -2.772083351973       -6.66       -3.19 

1.7735576926412076

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770705565464                   -0.53    8.0
  2   -2.772053211304       -2.87       -1.31    1.0
  3   -2.772082749166       -4.53       -2.57    1.0
  4   -2.772083404784       -6.18       -3.57    2.0
  5   -2.772083416362       -7.94       -3.94    2.0
  6   -2.772083417784       -8.85       -5.14    1.0
  7   -2.772083417810      -10.57       -5.86    2.0
  8   -2.772083417811      -12.79       -6.35    1.0
  9   -2.772083417811      -13.14       -7.12    2.0
 10   -2.772083417811   +  -14.88       -8.25    2.0

Polarizability via ForwardDiff:       1.7725349642569395
Polarizability via finite difference: 1.7735576926412076
